# Analysis of Potential Store Locations

## Introduction

This Jupyter Notebook analyzes potential store locations based on data obtained from loopnet.com and the census website.

### Goals
- Determine the suitability of potential store locations.
- Analyze the correlation between various factors and suitability.

## Data Overview
- **Data Sources:**
  - *Addresses:* Open addresses for rent obtained from loopnet.com.
  - *Existing Apple Stores Addresses:* obtained from apple.com.
  - *Income:* Income data obtained from the census website.

- **Initial Variables:**
  - `Address`: Address of the potential location.
  - `City`: City where the potential location is located.
  - `ZIP`: ZIP code of the potential location.
  - `Year Built`: Year the building was constructed.
  - `SF`: Square footage of the potential location.
  - `Price`: Rental price of the potential location.

- **Derived Variables:**
  - `potential_location`: Binary variable indicating suitability of a location.
  - `nearest_distance`: Distance to the nearest existing store.
  - `weighted_avg_income`: Weighted average income in the area.
  - `yearly_price_per_SF`: Yearly price per square foot.



## Libraries Import

In [432]:
import numpy as np
import pandas as pd
import geopandas as gpd
import re
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from IPython.display import IFrame
import seaborn as sns

## Function to Read Text File Content

In [433]:
def text_file_content(file_path):
    with open(file_path, 'r') as file:
        content = file.read().splitlines()
    return content

# Example usage:
file_path = 'nyc_rentals_ret.txt'  # loopnet addresses text file
lines_text = (text_file_content(file_path))
# Create a DataFrame
labels = ['Address', 'City and ZIP', 'Year Built', 'SF', 'Price']

def extract_info(text_lines):
    var_list = []
    for j in range(1,6):
        var_list.append([text_lines[i] for i in range(j,len(text_lines), 6)]) 
    return var_list

sublists_vals = (extract_info(lines_text))

## Display Initial Dataframe and Variables Info

In [434]:
df = pd.DataFrame(sublists_vals, index=labels).T
df

,Address,City and ZIP,Year Built,SF,Price
0,2586 Linden Blvd,"Brooklyn, NY, 11208",Built in 2015,"8,500 SF Retail Space",$48.00 SF/YR
1,103 Macdougal St,"New York, NY, 10012",Built in 1900,"5,000 - 12,000 SF Retail Spaces",$120.00 SF/YR
2,336 W 23rd St,"New York, NY, 10011",Built in 1910,"2,000 - 6,200 SF Retail Spaces",48.00 - $49.00 SF/YR
3,1 Wall Street,"New York, NY, 10005",Built in 1904,"5,000 - 12,500 SF Retail Spaces",84.00 - $88.00 SF/YR
4,31 W 21st St,"New York, NY, 10010",Built in 1908,"9,000 SF Retail Spaces",$115.00 SF/YR
...,...,...,...,...,...
506,625-649 Eighth Ave,"New York, NY, 10018",Built in 1950,"196 - 29,905 SF Retail Spaces",none
507,4473 Amboy Rd,"Staten Island, NY, 10312",Built in 2022,"2,000 - 15,000 SF Retail Space",none
508,143 Fulton St,"New York, NY, 10038",Built in 2018,"570 - 8,444 SF Retail Spaces",none
509,768 5th Ave,"New York, NY, 10019",Built in 1907,"440 - 32,940 SF Retail Spaces",none


In [ ]:
df.info()

The dataset consists of a DataFrame with 511 entries and 5 columns:

- **Address**: Object type, 511 non-null values.
- **City and ZIP**: Object type, 511 non-null values.
- **Year Built**: Object type, 511 non-null values.
- **SF**: Object type, 511 non-null values.
- **Price**: Object type, 511 non-null values.

It's important to note that these variables need to be converted to int or float for our analysis.


### Splitting 'City and ZIP' Column

We split the 'City and ZIP' column into 'City' and 'ZIP' columns since we need the ZIP code for further analysis.


In [ ]:
df[['City', 'ZIP']] = df['City and ZIP'].str.split('NY, ', expand=True)


In [ ]:
df.head(20)

# Data Preprocessing

We performed several preprocessing steps on the dataset:

1. Stripped whitespace from the 'Price' column.
2. Replaced 'none' values in the 'Price' column with '- 0 SF/YR'.
3. Extracted the yearly price per square foot ('yearly_price_per_SF') and square footage ('SF') from the 'Price' column.
4. Extracted the year built ('year_built') from the 'Year Built' column.
5. Removed extraneous information from the 'Address' column.

Finally, we dropped the unnecessary columns 'Price' and 'Year Built'.


In [ ]:
df['Price'] = df['Price'].str.strip()

# Replace 'none' with '- 0 SF/YR'
df['Price'] = df['Price'].replace('none', ' - 0 SF/YR')

# Step 2 & 3: Extract substring between '-' or '$' and 'SF/YR' in the 'Price' column
#df['yearly_price_per_SF'] = df['Price'].str.extract(r'(-|\$)(.*?)(?= SF/YR)', expand=False)
def get_substring(string, start_vals, end_vals):
    max_indices_from_left = ([string.rfind(val) for val in start_vals])
    max_index_from_left = max(max_indices_from_left) if max_indices_from_left else -1
    truncate_left_side = string[max_index_from_left+len(string[max_index_from_left]):]
    min_indices_from_right = [truncate_left_side.find(val) for val in end_vals]
    min_index_from_right = min(min_indices_from_right) if min_indices_from_right else -1
    truncate_right_side = truncate_left_side[:min_index_from_right]
    return truncate_right_side #left_side #right_side[0] if right_side else left_side


df['yearly_price_per_SF'] = df['Price'].apply(lambda x: get_substring(x, ['$', '-'], ['SF/YR']))
df['SF'] = df['SF'].apply(lambda x: get_substring(x, ['-'], ['SF']))

df['year_built'] = df['Year Built'].str.replace('Built in ', '')

def remove_info(s):
    return re.sub(r'^.*?-', '', s)


df['Address'] = df['Address'].apply(remove_info)

# Step 4: Drop unnecessary columns
df.drop(columns=[ 'Price', 'Year Built'], inplace=True)

df['full_address'] = df['Address'] + ' ' + df['City and ZIP']
df.head(20)

In [ ]:
'''df['full_address'] = df['Address'] + ' ' + df['City and ZIP']
df.head(20)'''

### Convert Text Coordinates to Tuples

The function below takes a list of string coordinates and converts them into a list of tuple coordinates. It reads coordinates from a text file and parses them using the `ast.literal_eval` function. The resulting list contains tuples representing the coordinates.

**Parameters:**
- `string_coords_from_text` (list): A list of string coordinates.

**Returns:**
- `coords_from_text_tuples` (list): A list of tuple coordinates.

Example usage:
```python
coords_from_text = text_file_content('geospatial_coords_nyc.txt')
coord_tuples = text_to_coords(coords_from_text)


In [ ]:
import ast

# Read coordinates from the text file
coords_from_text = (text_file_content('geospatial_coords_nyc.txt'))

# Convert string coordinates to tuples
def text_to_coords(string_coords_from_text):
    last_coord = [ast.literal_eval(string_coords_from_text[-1])]
    coords_from_text_tuples = [ast.literal_eval(str_coord)[0]
                               for str_coord in coords_from_text][:-1]+last_coord
    return coords_from_text_tuples

### Extracting Coordinates from Text and Adding to DataFrame 

The code snippet below extracts coordinates from a text file using the `text_to_coords` function and adds them to a DataFrame. The `text_to_coords` function converts string coordinates to tuple coordinates, which are then unpacked into separate lists for latitude and longitude. These lists are then added as new columns 'latitude' and 'longitude' to the DataFrame.



In [ ]:
coords_from_text_tuples = text_to_coords(coords_from_text)

coords_list_lat, coords_list_long = list(map(list, zip(*coords_from_text_tuples)))
df['latitude'] = coords_list_lat
df['longitude'] = coords_list_long
df

### Processing NYC Income Data 

Here below the NYC income data from an Excel file is extracted and converted into a pandas DataFrame. It then preprocesses the data by removing rows with missing values, creating a new column 'avg_income' with specified values based on income ranges, and converting the 'ZIP' column to string type while removing everything after '.'.

Next, it filters the DataFrame to keep only the rows where the ZIP code is in the NYC list, as many ZIP codes are from the entire state of New York. It calculates the total income for each ZIP code by multiplying the average income by the number of earners and groups the data by ZIP code, summing the total income and total number of earners.

Finally, it calculates the weighted average income for each ZIP code and resets the index to make 'zip_code' a column again.


In [ ]:
# Read the nyc income Excel file into a pandas DataFrame
df_income = pd.read_excel(r'nyc_income_by_zip.xlsx')
# Remove rows with missing values (NaN)
df_income.dropna(inplace=True)
# Create a new column 'avg_income' with specified values (repeating pattern) based on the provided income range
df_income['avg_income'] = [17, 37, 63, 87, 150, 250] * 1535
# Convert 'zip_code' column to string type and remove everything after '.'
df_income['ZIP'] = df_income['ZIP'].astype(str).apply(lambda x: x.split('.')[0])
# Filter the DataFrame to keep rows where the zip_code is in the nyc list (plenty are from the whole state of NY)
df_income = df_income[df_income['ZIP'].isin(list(df.ZIP))] 
# Calculate the total income for each zip code by multiplying the average income by the number of earners
df_income['total_income'] = df_income['avg_income'] * df_income['nr_earners']
# Group by 'zip_code' and sum the total income and total number of earners
grouped_data = df_income.groupby('ZIP').agg({'total_income': 'sum', 'nr_earners': 'sum'})
# Calculate the weighted average income for each zip code
grouped_data['weighted_avg_income'] = grouped_data['total_income'] / grouped_data['nr_earners']
# Reset index to make 'zip_code' a column again
grouped_data = grouped_data.reset_index()
grouped_data

In [ ]:
# Filter the DataFrame to keep rows where the zip_code is in the nyc list (plenty are from the whole state of NY)
df = df[df['ZIP'].isin(list(grouped_data.ZIP))] 
# Merge the new DataFrame with the old DataFrame on the zip code column
df = pd.merge(df, grouped_data[['ZIP', 'weighted_avg_income']], on='ZIP', how='left')
df

### Calculating Nearest Distance to Apple Stores 

The code snippet below calculates the nearest distance from each coordinate in a DataFrame to a list of Apple store coordinates. It first reads the Apple store coordinates from a text file and converts them into a cleaned list of tuples.

Then, it iterates over each coordinate in the DataFrame and calculates the distance to each Apple store coordinate using the geodesic distance method from the geopy library. The minimum distance is updated if necessary.

Finally, it assigns the minimum distance to a new column named 'nearest_distance' in the DataFrame and displays the DataFrame with the nearest distances.


In [ ]:
from geopy.distance import geodesic

# Read coordinates from the text file
apple_stores_coords = (text_file_content('apple_store_locs_nyc.txt'))
last_coord = [ast.literal_eval(apple_stores_coords[-1])]
apple_stores_coords_from_text_tuples = [ast.literal_eval(str_coord)[0] for str_coord in apple_stores_coords][:-1]+last_coord


# Assume your DataFrame containing coordinates is named df
# Iterate over each coordinate in your DataFrame
for index, row in df.iterrows():
    min_distance = float('inf')  # Initialize with infinity
    # Iterate over each coordinate from the text file
    for coord in apple_stores_coords_from_text_tuples:
        # Calculate distance between coordinates
        distance = geodesic((row['latitude'], row['longitude']), coord).meters
        # Update minimum distance if necessary
        min_distance = min(min_distance, distance)
    # Assign minimum distance to a new column in your DataFrame
    df.at[index, 'nearest_distance'] = min_distance

# Display DataFrame with nearest distances
df

In [ ]:
# Remove non-numeric characters and convert to float
df['SF'] = df['SF'].str.replace(',', '').str.strip().astype(float)

# convert price to float
df['yearly_price_per_SF'] = df['yearly_price_per_SF'].astype(float)

# Convert 'year_built' column to integers
df['year_built'] = df['year_built'].astype(int)

### Creating Binary Variable for Potential Location 

Here we create a binary variable named 'potential_location' in the DataFrame based on specified criteria. 

1. The distance to the nearest Apple store is compared to the 25th percentile distance among all locations.
2. The weighted average income in the area is compared to the 25th percentile income among all locations.
3. The square footage of the potential location is compared to the 25th percentile square footage among all locations.

These criteria are combined using logical AND operations to determine potential locations. If a location satisfies all criteria, it is labeled as a potential location with a value of 1; otherwise, it is labeled as 0. The resulting binary variable is added to the DataFrame.


In [ ]:
# Create binary variables based on criteria
potential_location_distance = df['nearest_distance'] >= np.quantile(df.nearest_distance, .25)
potential_location_income = df['weighted_avg_income'] >= np.quantile(df.weighted_avg_income, 0.25)
potential_location_sf = df['SF'] >= np.quantile(df.SF, 0.25)
#potential_location_price = df['yearly_price_per_SF'] <= np.quantile(df.yearly_price_per_SF, .95)
#potential_location_year = df['year_built'] >= np.median(df.year_built)

# Combine criteria
potential_location_combined = (
    potential_location_distance & 
    potential_location_income & 
    potential_location_sf 
    #potential_location_price 
   #& potential_location_year
)

# Label dataframe based on combined criteria
df['potential_location'] = potential_location_combined.astype(int)
df

### Creating and Displaying Map of Potential Locations 

The map below visualizes the feasible and unfeasible locations for opening new Apple stores. 

1. The map is centered at the coordinates of New York City with a zoom level of 10.
2. A MarkerCluster layer is added to group markers for potential locations.
3. Markers are added to the map for each potential location in the DataFrame. If a location is labeled as a potential location (with a value of 1 in the 'potential_location' column), a green check icon is used; otherwise, a red cross icon is used.
4. The map is saved as an HTML file named 'potential_locations_map.html'.
5. Finally, the map is displayed directly in the Jupyter Notebook using an IFrame.


In [ ]:
map_center = [40.7128, -74.0060]  # New York City coordinates
map_zoom = 10  # Zoom level
m = folium.Map(location=map_center, zoom_start=map_zoom)

# MarkerCluster layer
marker_cluster = MarkerCluster().add_to(m)

# Add markers for potential locations
for index, row in df.iterrows():
    if row['potential_location'] == 1:
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"Location: {index}",
            icon=folium.Icon(color='green', icon='check')
        ).add_to(marker_cluster)
    else:
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"Location: {index}",
            icon=folium.Icon(color='red', icon='times')
        ).add_to(marker_cluster)

# Save the map as an HTML file
m.save('potential_locations_map.html')

# Display the map directly in the notebook
IFrame(src='potential_locations_map.html', width=700, height=600)



### distribution of predictors across the target variable via histograms.

1. The number of rows and columns for subplots is determined based on the number of predictors. Two additional rows are added to account for the legend and title.
2. Subplots are created using the `plt.subplots` function with the specified number of rows and columns.
3. The axes are flattened for easy iteration.
4. Each predictor is plotted individually on a subplot using Seaborn's `histplot` function.
5. To balance the classes of the target variable, the dataframe is randomly sampled. Half of the samples with the target variable equal to 0 are selected, and all samples with the target variable equal to 1 are included.
6. Titles, x-labels, y-labels, and legends are set for each subplot.

In [ ]:
import numpy as np

# Determine the number of rows and columns for subplots
num_rows = (len(predictors) + 2) // 3  # Add 2 to account for the legend and title
num_cols = min(len(predictors), 3)

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plotting each predictor individually
for i, predictor in enumerate(predictors):
    ax = axes[i]
    # Randomly sample the dataframe to balance the target variable classes
    df_sampled = pd.concat([
        df[df[target] == 0].sample(frac=0.5),
        df[df[target] == 1].sample(frac=1)
    ])
    sns.histplot(data=df_sampled, x=predictor, hue=target, multiple='stack', bins=25, ax=ax)
    ax.set_title(f'Distribution of {target} Across {predictor}', fontsize=6)  # Adjust the title fontsize
    ax.set_xlabel(predictor)
    ax.set_ylabel('Count')
    ax.legend(title=target)

# Remove empty subplots
for i in range(len(predictors), num_rows * num_cols):
    fig.delaxes(axes[i])

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


The histogram analysis reveals an interesting trend in the distribution of categories across different frequencies. Specifically, as the frequency of occurrences increases, the distribution tends to favor category 0 slightly more. Conversely, when the frequency decreases, category 1 becomes more prevalent in the distribution.


In [ ]:
selected_columns = predictors  # Replace with the column names you want to select
filtered_df = df[selected_columns]
filtered_df.info()

###  Machine Learning Algorithms Selection and Motivation:

1. **Logistic Regression**:
   - **Motivation**: Logistic Regression is a simple and interpretable algorithm that is well-suited for binary classification tasks.
   - **Strengths**: Easy to implement, computationally efficient, provides probabilities for predictions, interpretable coefficients.
   - **Use Cases**: Often used as a baseline model for binary classification tasks, especially when interpretability is important.

2. **Random Forest**:
   - **Motivation**: Random Forest is an ensemble learning method that builds multiple decision trees and combines their predictions to improve accuracy and reduce overfitting.
   - **Strengths**: High accuracy, robust to overfitting, handles both numerical and categorical data, provides feature importances.
   - **Use Cases**: Commonly used for classification and regression tasks where high accuracy is desired, and interpretability is not a primary concern.

3. **Support Vector Machine (SVM)**:
   - **Motivation**: SVM is a powerful algorithm for classification tasks, particularly when dealing with complex, nonlinear decision boundaries.
   - **Strengths**: Effective in high-dimensional spaces, versatile (can use different kernel functions), robust to overfitting in high-dimensional spaces.
   - **Use Cases**: Used for both classification and regression tasks, especially when the data is not linearly separable or when dealing with small to medium-sized datasets.

4. **Gradient Boosting**:
   - **Motivation**: Gradient Boosting is an ensemble learning technique that builds a strong predictive model by combining multiple weak models (typically decision trees) sequentially.
   - **Strengths**: High predictive accuracy, handles complex relationships between features, robust to outliers, automatically handles missing data.
   - **Use Cases**: Widely used in both regression and classification tasks, particularly when dealing with structured/tabular data and when model interpretability is not a primary concern.

5. **Multilayer Perceptron (MLP)**:
   - **Motivation**: MLP is a type of artificial neural network composed of multiple layers of nodes (neurons) that can learn complex patterns in data.
   - **Strengths**: Can capture complex relationships in data, suitable for nonlinear problems, performs well with large datasets, can learn feature hierarchies automatically.
   - **Use Cases**: Used for a wide range of tasks, including classification, regression, and pattern recognition


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Split data into features (X) and target (y)
X = filtered_df
y = df['potential_location']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
logreg = LogisticRegression()
rf = RandomForestClassifier()
svm = SVC()
gbm = GradientBoostingClassifier()
mlp = MLPClassifier()

# Train models
logreg.fit(X_train, y_train)
rf.fit(X_train, y_train)
svm.fit(X_train, y_train)
gbm.fit(X_train, y_train)
mlp.fit(X_train, y_train)

# Make predictions
logreg_preds = logreg.predict(X_test)
rf_preds = rf.predict(X_test)
svm_preds = svm.predict(X_test)
gbm_preds = gbm.predict(X_test)
mlp_preds = mlp.predict(X_test)

# Evaluate performance
models = [logreg, rf, svm, gbm, mlp]
model_names = ['Logistic Regression', 'Random Forest', 'SVM', 'Gradient Boosting', 'MLP']



for model, name in zip(models, model_names):
    preds_train, preds_test = model.predict(X_train), model.predict(X_test) 
    accuracy_train, accuracy_test = accuracy_score(y_train, preds_train), accuracy_score(y_test, preds_test)
    precision_train, precision_test = precision_score(y_train, preds_train), precision_score(y_test, preds_test)
    recall_train, recall_test = recall_score(y_train, preds_train), recall_score(y_test, preds_test)
    f1_train, f1_test = f1_score(y_train, preds_train), f1_score(y_test, preds_test)
    roc_auc_train, roc_auc_test = roc_auc_score(y_train, preds_train), roc_auc_score(y_test, preds_test) 
    
    print(f"Performance metrics for {name}:")
    print("accuracy train {tr}, accuracy test {test}".format(tr= accuracy_train, test = accuracy_test))
    print("precision train {tr}, precision test {test}".format(tr= precision_train, test = accuracy_test))
    print("recall train {tr}, recall test {test}".format(tr= recall_train, test = recall_test))
    print("f1 train {tr}, f1 test {test}".format(tr= f1_train, test = recall_test))
    print("roc auc train {tr}, roc auc test {test}".format(tr= roc_auc_train, test = recall_test))
    print()
 


### Analysis of Model Performance Metrics

#### Logistic Regression:
- **Accuracy:** 
  - Train: 0.6972
  - Test: 0.6566
- **Precision:** 
  - Train: 0.6712
  - Test: 0.6566
- **Recall:** 
  - Train: 0.3403
  - Test: 0.2059
- **F1 Score:** 
  - Train: 0.4516
  - Test: 0.2059
- **ROC AUC:** 
  - Train: 0.6220
  - Test: 0.2059

Logistic Regression achieves moderate accuracy but has relatively low precision and recall, indicating that it may not effectively identify potential locations. The F1 score suggests a balance between precision and recall, but the ROC AUC score indicates only slightly better-than-random performance in distinguishing between potential and non-potential locations.

#### Random Forest:
- **Accuracy:** 
  - Train: 1.0000
  - Test: 1.0000
- **Precision:** 
  - Train: 1.0000
  - Test: 1.0000
- **Recall:** 
  - Train: 1.0000
  - Test: 1.0000
- **F1 Score:** 
  - Train: 1.0000
  - Test: 1.0000
- **ROC AUC:** 
  - Train: 1.0000
  - Test: 1.0000

Random Forest achieves perfect scores across all metrics, indicating flawless performance

#### Support Vector Machine (SVM):
- **Accuracy:** 
  - Train: 0.7048
  - Test: 0.6768
- **Precision:** 
  - Train: 0.7917
  - Test: 0.6768
- **Recall:** 
  - Train: 0.2639
  - Test: 0.1176
- **F1 Score:** 
  - Train: 0.3958
  - Test: 0.1176
- **ROC AUC:** 
  - Train: 0.6119
  - Test: 0.1176

SVM achieves moderate accuracy and precision, but its recall is quite low, indicating that it fails to identify many potential locations. The F1 score is also relatively low, suggesting a trade-off between precision and recall. The ROC AUC score indicates slightly better-than-random performance.

#### Gradient Boosting:
- **Accuracy:** 
  - Train: 1.0000
  - Test: 1.0000
- **Precision:** 
  - Train: 1.0000
  - Test: 1.0000
- **Recall:** 
  - Train: 1.0000
  - Test: 1.0000
- **F1 Score:** 
  - Train: 1.0000
  - Test: 1.0000
- **ROC AUC:** 
  - Train: 1.0000
  - Test: 1.0000

Gradient Boosting achieves perfect scores across all metrics, together with Random Forest it is so far the best model of the tried ones 

#### Multilayer Perceptron (MLP):
- **Accuracy:** 
  - Train: 0.7099
  - Test: 0.6869
- **Precision:** 
  - Train: 0.8750
  - Test: 0.6869
- **Recall:** 
  - Train: 0.2431
  - Test: 0.1471
- **F1 Score:** 
  - Train: 0.3804
  - Test: 0.1471
- **ROC AUC:** 
  - Train: 0.6115
  - Test: 0.1471

MLP achieves good accuracy and precision, with moderate recall. The F1 score indicates a balance between precision and recall, and the ROC AUC score suggests reasonable discrimination ability. However, the model's performance on recall and F1 score is lower compared to the gradient boosting especially when considering the F1 score since the accuracy is not totally reliable given the unbalanced distribution of the target variable.


Here is a visual representation of some of the accuracy metrics for each ML model. It's easy to detect that both Random Forest and Gradient Boosting performed impeccably given the absence of false positives and false negatives

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Train each model and make predictions
model_names = ['Logistic Regression', 'Random Forest', 'SVM', 'Gradient Boosting', 'MLP']

# Create subplots with 3 rows and 2 columns
fig, axes = plt.subplots(3, 2, figsize=(12, 12))

# Flatten the axes array for easy iteration
axes = axes.flatten()

for i, (model, name) in enumerate(zip(models, model_names)):
    preds_test = model.predict(X_test)
    cm = confusion_matrix(y_test, preds_test)
    
    # Plot confusion matrix
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', ax=axes[i])
    axes[i].set_title(f'Confusion Matrix - {name}')
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('True')

# Remove any extra empty subplots
for j in range(len(model_names), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()



